In [2]:
import numpy as np
import plotly.graph_objects as go
import quantfreedom as qf
from plotly.subplots import make_subplots

from quantfreedom.plotting.plot_helper_functions import plot_results_candles_and_chart

In [3]:
price_data = qf.generate_candles(
    number_of_candles=1000,
    plot_candles=False,
    seed=7,
)

In [4]:
rsi_data = qf.from_talib(
    func_name="rsi",
    price_data=price_data,
    timeperiod=20,
    plot_results=True,
    price="low",
)

In [5]:
plot_results_candles_and_chart(
    ta_lib_data=rsi_data.loc["2001-08-10":"2001-11-10"],
    price_data=price_data.loc["2001-08-10":"2001-11-10"],
)


In [6]:
df = price_data.droplevel(level=0, axis=1).loc['2001-08-10': '2001-11-10']
df['rsi'] = rsi_data['QuantFreedom'][20].loc['2001-08-10': '2001-11-10']

df['piv_low'] = df['low'].rolling(window=20).min().replace(np.nan, 0)

df['piv_low_shift'] = df['piv_low'].shift(1).replace(np.nan, 0)

df['piv_point_graph'] = np.where(
    df['piv_low'].values == df['piv_low_shift'].values, np.nan, df['low'].values)
df['piv_point_graph'] = np.where(
    df['close'].values > df['open'].values, np.nan, df['piv_point_graph'].values)

df['rsi_low'] = df['rsi'].rolling(window=20).min().replace(np.nan, 0)

df['rsi_low_shift'] = df['rsi_low'].shift(1).replace(np.nan, 0)

df['rsi_low_graph'] = np.where(
    df['rsi_low'].values == df['rsi_low_shift'].values, np.nan, df['rsi'].values)

In [7]:
fig = make_subplots(
    rows=2,
    cols=1,
    row_heights=[0.7, 0.3],
    shared_xaxes=True,
    vertical_spacing=0.02,
)
fig.append_trace(
    go.Candlestick(
        x=df.index,
        open=df["open"],
        high=df["high"],
        low=df["low"],
        close=df["close"],
    ),
    row=1,
    col=1,
)

fig.add_scatter(
    x=df.index,
    y=df["piv_point_graph"].values - 2,
    mode="markers",
    marker=dict(size=4, color="MediumPurple"),
    name="pivot",
    row=1,
    col=1,
)

fig.append_trace(
    go.Scatter(x=df.index, y=df["rsi"]),
    row=2,
    col=1,
)
fig.add_scatter(
    x=df.index,
    y=df["rsi_low_graph"].values - 2,
    mode="markers",
    marker=dict(size=4, color="MediumPurple"),
    name="pivot",
    row=2,
    col=1,
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    height=800,
)
fig.show()

In [8]:
df

candle_info,open,high,low,close,rsi,piv_low,piv_low_shift,piv_point_graph,rsi_low,rsi_low_shift,rsi_low_graph
open_time,,,,,,,,,,,
2001-08-10,"29,836.84","29,837.15","29,830.29","29,835.45",45.91,0.00,0.00,NaN,0.00,0.00,NaN
2001-08-11,"29,835.03","29,839.39","29,830.23","29,831.61",45.88,0.00,0.00,NaN,0.00,0.00,NaN
2001-08-12,"29,830.45","29,830.46","29,823.03","29,823.63",42.22,0.00,0.00,NaN,0.00,0.00,NaN
2001-08-13,"29,822.73","29,829.96","29,821.68","29,827.88",41.56,0.00,0.00,NaN,0.00,0.00,NaN
2001-08-14,"29,827.82","29,827.82","29,816.23","29,822.66",38.99,0.00,0.00,NaN,0.00,0.00,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2001-11-06,"29,822.95","29,829.29","29,822.95","29,827.33",64.63,"29,791.13","29,791.13",NaN,47.40,47.40,NaN
2001-11-07,"29,826.71","29,834.78","29,826.71","29,830.77",66.41,"29,791.13","29,791.13",NaN,47.40,47.40,NaN
2001-11-08,"29,831.06","29,831.84","29,816.83","29,819.02",58.26,"29,791.13","29,791.13",NaN,47.40,47.40,NaN


In [57]:
price_low_values = df.low.values
price_close_values = df.close.values
price_open_values = df.open.values

piv_low_index = 0
piv_low = price_low_values[0]
piv_low_graph = np.full_like(price_low_values, np.nan)
piv_low_graph[0] = piv_low

rsi_values = df.rsi.values
divergence_index = np.full_like(price_low_values, -1, dtype=np.int_)
rsi_line_graph = np.full_like(price_low_values, np.nan)

candle_line_graph = np.full_like(price_low_values, np.nan)
candle_div_graph_lb = np.full_like(price_low_values, np.nan)
candle_div_graph_start = np.full_like(price_low_values, np.nan)

lookback = 20
plot_index = df.index
line_counter = 0

for x in range(1, price_low_values.size):
    if price_low_values[x] < piv_low and price_close_values[x] < price_open_values[x]:
        for lb in range(1, lookback):
            if x - lb > 0:
                lb_index = x - lb
                if (
                    rsi_values[x] > rsi_values[lb_index]
                    and price_close_values[lb_index] < price_open_values[lb_index]
                ):
                    candle_line_graph[line_counter + 1] = price_low_values[lb_index] - 2
                    candle_line_graph[line_counter] = price_low_values[x] - 2

                    rsi_line_graph[line_counter + 1] = rsi_values[lb_index] - 2
                    rsi_line_graph[line_counter] = rsi_values[x] - 2

                    divergence_index[line_counter + 1] = lb_index
                    divergence_index[line_counter] = x

                    line_counter += 2

                    break
            else:
                break
        piv_low = price_low_values[x]
        piv_low_graph[x] = price_low_values[x] - 2
        changed_pivot = True
    
    # keep some type of counter that checks if we have gone 20 candles without a new low
rsi_line_graph = rsi_line_graph[:line_counter]
candle_line_graph = candle_line_graph[:line_counter]
divergence_index = divergence_index[:line_counter]
divergence_index = plot_index[divergence_index.tolist()]

In [58]:
fig = make_subplots(
    rows=2,
    cols=1,
    row_heights=[0.7, 0.3],
    shared_xaxes=True,
    vertical_spacing=0.02,
)
fig.append_trace(
    go.Candlestick(
        x=plot_index,
        open=df["open"],
        high=df["high"],
        low=df["low"],
        close=df["close"],
    ),
    row=1,
    col=1,
)

fig.add_scatter(
    x=plot_index,
    y=piv_low_graph,
    mode="markers",
    marker=dict(size=4, color="MediumPurple"),
    name="candle pivot",
    row=1,
    col=1,
)

fig.append_trace(
    go.Scatter(
        x=plot_index,
        y=rsi_values,
        name="RSI",
    ),
    row=2,
    col=1,
)
start = 0
end = 2
for x in range(rsi_line_graph.size):
    fig.add_scatter(
        x=divergence_index[start:end],
        y=candle_line_graph[start:end],
        mode="markers+lines",
        marker=dict(size=10, symbol="arrow-up"),
        name="Candle Line Div",
        row=1,
        col=1,
    )
    fig.add_scatter(
        x=divergence_index[start:end],
        y=rsi_line_graph[start:end],
        mode="markers+lines",
        marker=dict(size=10, symbol="arrow-up"),
        name="RSI div",
        row=2,
        col=1,
    )
    start = end
    end += 2
fig.update_layout(
    xaxis_rangeslider_visible=False,
    height=800,
)
fig.show()